In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import metrics
from sklearn import ensemble

In [2]:
matches = pd.read_csv('new_data/tenis.csv')
matches = matches[matches.year > 2016]

c:\users\aleksandra.d\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
matches = matches.sort_values(by='tourney_date')

In [4]:
#broj pobeda pre meca za oba protivnika
def add_score_before_that_game(df):
    df['winner_wins'] = 0
    df['loser_wins'] = 0
    df['winner_loses'] = 0
    df['loser_loses'] = 0
    winners = dict()
    losers = dict()
    for ind, row in df.iterrows():
        winner = row['winner_id']
        loser  = row['loser_id']
        if winner not in winners:
            winners[winner] = 0
        else:
            winners[winner] += 1
        if loser not in winners:
            winners[loser] = 0
        if loser not in losers:
            losers[loser] = 0
        else:
            losers[loser] += 1
        if winner not in losers:
            losers[winner] = 0
        df.at[ind, 'winner_wins'] = winners[winner]
        df.at[ind, 'loser_wins'] = winners[loser]
        df.at[ind, 'winner_loses'] = losers[winner]
        df.at[ind, 'loser_loses'] = losers[loser]
    return df

In [5]:
matches = add_score_before_that_game(matches)

In [6]:
#medjusobni skor pre meca
def add_h2h_before_that_game(df):
    df['winner_h2h'] = 0
    df['loser_h2h'] = 0
    head_to_heads = dict()
    for ind, row in df.iterrows():
        winner = row['winner_id']
        loser  = row['loser_id']
        head_to_head = str(winner) + ' vs ' + str(loser)
        head_to_head1 = str(loser) + ' vs ' + str(winner)
        if head_to_head not in head_to_heads:
            if head_to_head1 not in head_to_heads:
                head_to_heads[head_to_head] = {}
                head_to_heads[head_to_head][winner] = 0
                head_to_heads[head_to_head][loser] = 0
            else: 
                head_to_heads[head_to_head1][winner] += 1
                head_to_head = head_to_head1
        else:
            head_to_heads[head_to_head][winner] += 1
        df.at[ind, 'winner_h2h'] = head_to_heads[head_to_head][winner]
        df.at[ind, 'loser_h2h'] = head_to_heads[head_to_head][loser]
    return df

In [7]:
matches = add_h2h_before_that_game(matches)

In [8]:
#broj pobeda pre meca na podlozi na koji se igra za oba protivnika
def add_score_surface_before_that_game(df):
    df['winner_surface_wins'] = 0
    df['loser_surface_wins'] = 0
    df['winner_surface_games'] = 0
    df['loser_surface_games'] = 0
    player_surface = dict()
    for ind, row in df.iterrows():
        winner = row['winner_id']
        loser  = row['loser_id']
        surface = row['surface']
        if winner not in player_surface:
            player_surface[winner] = {'Hard_w': 0, 'Carpet_w': 0, 'Clay_w': 0, 'Grass_w': 0,'Hard_p': 0, 'Carpet_p': 0, 'Clay_p': 0, 'Grass_p': 0}
        else:
            player_surface[winner][surface + '_w'] += 1
            player_surface[winner][surface + '_p'] += 1
        if loser not in player_surface:
            player_surface[loser] = {'Hard_w': 0, 'Carpet_w': 0, 'Clay_w': 0, 'Grass_w': 0,'Hard_p': 0, 'Carpet_p': 0, 'Clay_p': 0, 'Grass_p': 0}
        else:  
            player_surface[loser][surface + '_p'] += 1
        df.at[ind, 'winner_surface_wins'] = player_surface[winner][surface + '_w']
        df.at[ind, 'winner_surface_games'] = player_surface[winner][surface + '_p']
        df.at[ind, 'loser_surface_wins'] = player_surface[loser][surface + '_w']
        df.at[ind, 'loser_surface_games'] = player_surface[loser][surface + '_p']
    return df

In [9]:
matches = add_score_surface_before_that_game(matches)

In [10]:
#broj pobeda te godine pre meca za oba protivnika
def add_year_wins_before_that_game(df):
    df['winner_year_wins'] = 0
    df['loser_year_wins'] = 0
    df['winner_year_games'] = 0
    df['loser_year_games'] = 0
    player_year = dict()
    for ind, row in df.iterrows():
        winner = row['winner_id']
        loser  = row['loser_id']
        year = str(row['year'])
        if winner not in player_year:
            player_year[winner] = {}
            player_year[winner][year + '_w'] = 0 
            player_year[winner][year + '_g'] = 0 
        else:
            if year + '_w' in player_year[winner]:
                player_year[winner][year + '_w'] += 1
                player_year[winner][year + '_g'] += 1
            else:
                player_year[winner][year + '_w'] = 0
                player_year[winner][year + '_g'] = 0
        if loser not in player_year:
            player_year[loser] = {}
            player_year[loser][year + '_g'] = 0 
            player_year[loser][year + '_w'] = 0 
        else: 
            if year + '_w' in player_year[loser]:
                player_year[loser][year + '_g'] += 1
                player_year[loser][year + '_w'] = 0
            else:
                player_year[loser][year + '_g'] = 0
                player_year[loser][year + '_w'] = 0
        df.at[ind, 'winner_year_wins'] = player_year[winner][year + '_w']
        df.at[ind, 'winner_year_games'] = player_year[winner][year + '_g']
        df.at[ind, 'loser_year_wins'] = player_year[loser][year + '_w']
        df.at[ind, 'loser_year_games'] = player_year[loser][year + '_g']
    return df

In [11]:
matches = add_year_wins_before_that_game(matches)

In [12]:
#broj pobeda na tom turniru pre meca za oba protivnika
def add_tournament_wins_before_that_game(df):
    df['winner_tournament_wins'] = 0
    df['loser_tournament_wins'] = 0
    df['winner_tournament_games'] = 0
    df['loser_tournamnet_games'] = 0
    player_tournament = dict()
    for ind, row in df.iterrows():
        winner = row['winner_id']
        loser  = row['loser_id']
        tournament = (row['tourney_name'])
        if winner not in player_tournament:
            player_tournament[winner] = {}
            player_tournament[winner][tournament + '_w'] = 0 
            player_tournament[winner][tournament + '_g'] = 0 
        else:
            if tournament + '_w' in player_tournament[winner]:
                player_tournament[winner][tournament + '_w'] += 1
                player_tournament[winner][tournament + '_g'] += 1
            else:
                player_tournament[winner][tournament + '_w'] = 0
                player_tournament[winner][tournament + '_g'] = 0
        if loser not in player_tournament:
            player_tournament[loser] = {}
            player_tournament[loser][tournament + '_g'] = 0 
            player_tournament[loser][tournament + '_w'] = 0 
        else: 
            if  tournament + '_w' in player_tournament[loser]:
                player_tournament[loser][tournament + '_g'] += 1
                player_tournament[loser][tournament + '_w'] = 0
            else:
                player_tournament[loser][tournament + '_g'] = 0
                player_tournament[loser][tournament + '_w'] = 0
        df.at[ind, 'winner_tournament_wins'] = player_tournament[winner][tournament + '_w']
        df.at[ind, 'winner_tournament_games'] = player_tournament[winner][tournament + '_g']
        df.at[ind, 'loser_tournament_wins'] = player_tournament[loser][tournament + '_w']
        df.at[ind, 'loser_tournament_games'] = player_tournament[loser][tournament + '_g']
    return df

In [13]:
matches = add_tournament_wins_before_that_game(matches)

In [14]:
#broj pobeda poslednjih 10 meceva pre meca za oba protivnika
def add_last_10_games_before_that_game(df):
    df['winner_10_wins'] = 0
    df['loser_10_wins'] = 0
    player_10 = dict()
    for ind, row in df.iterrows():
        winner = row['winner_id']
        loser  = row['loser_id']
        if winner not in player_10:
            player_10[winner] = [0,0,0,0,0,0,0,0,0,0]
            player_10[winner].pop(0)
            player_10[winner].append(1)
        else:
            player_10[winner].pop(0)
            player_10[winner].append(1)
        if loser not in player_10:
            player_10[loser] = [0,0,0,0,0,0,0,0,0,0]
        else:
            player_10[loser].pop(0)
            player_10[loser].append(0)
        df.at[ind, 'winner_10_wins'] = sum(player_10[winner])
        df.at[ind, 'loser_10_wins'] = sum(player_10[loser])
    return df

In [15]:
matches = add_last_10_games_before_that_game(matches)

In [16]:
#statistika
def add_statistic_about_games(df):
    df['winner_win%'] = df['winner_wins'] / (df['winner_wins'] + df['winner_loses']) 
    df['loser_win%'] = df['loser_wins'] / (df['loser_wins'] + df['loser_loses']) 
    winner_wins = df['winner_win%'].mean()
    loser_wins = df['loser_win%'].mean()
    df['winner_win%'].fillna(winner_wins, inplace = True)
    df['loser_win%'].fillna(loser_wins, inplace = True)
    df['winner_h2h%'] = df['winner_h2h'] / (df['winner_h2h'] + df['loser_h2h']) 
    df['winner_surface%'] = df['winner_surface_wins'] / df['winner_surface_games']
    df['loser_surface%'] = df['loser_surface_wins'] / df['loser_surface_games']
    winner_h2h = df['winner_h2h%'].mean()
    winner_surface = df['winner_surface%'].mean()
    loser_surface = df['loser_surface%'].mean()
    df['winner_h2h%'].fillna(winner_h2h, inplace = True)
    df['winner_surface%'].fillna(winner_surface, inplace = True)
    df['loser_surface%'].fillna(loser_surface, inplace = True)
    df['winner_year%'] = df['winner_year_wins'] / df['winner_year_games']
    df['loser_year%'] = df['loser_year_wins'] / df['loser_year_games']
    winner_year = df['winner_year%'].mean()
    loser_year = df['loser_year%'].mean()
    df['winner_year%'].fillna(winner_year, inplace = True)
    df['loser_year%'].fillna(loser_year, inplace = True)
    df['winner_tournament%'] = df['winner_tournament_wins'] / df['winner_tournament_games']
    df['loser_tournament%'] = df['loser_tournament_wins'] / df['loser_tournament_games']
    winner_tournament = df['winner_tournament%'].mean()
    loser_tournament = df['loser_tournament%'].mean()
    df['winner_tournament%'].fillna(winner_tournament, inplace = True)
    df['loser_tournament%'].fillna(loser_tournament, inplace = True)
    df['winner_10%'] = df['winner_10_wins'] / 10
    df['loser_10%'] = df['loser_10_wins'] / 10
    return df

In [17]:
matches = add_statistic_about_games(matches)

In [18]:
#kolona da li je bolje rankirani pobedio
matches['better_rank_win'] = matches['winner_rank'] < matches['loser_rank']

In [19]:
matches.columns

Index(['tourney_id', 'tourney_name', 'surface', 'draw_size', 'tourney_level',
       'tourney_date', 'match_num', 'winner_id', 'winner_seed', 'winner_entry',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_seed', 'loser_entry', 'loser_name', 'loser_hand',
       'loser_ht', 'loser_ioc', 'loser_age', 'score', 'best_of', 'round',
       'minutes', 'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced', 'l_ace', 'l_df', 'l_svpt',
       'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced',
       'winner_rank', 'winner_rank_points', 'loser_rank', 'loser_rank_points',
       'year', 'winner_wins', 'loser_wins', 'winner_loses', 'loser_loses',
       'winner_h2h', 'loser_h2h', 'winner_surface_wins', 'loser_surface_wins',
       'winner_surface_games', 'loser_surface_games', 'winner_year_wins',
       'loser_year_wins', 'winner_year_games', 'loser_year_games',
       'wi

In [20]:
matches_train = matches[matches.year !=2021]
matches_test =  matches[matches.year ==2021]

In [25]:
X_train = matches_train[['winner_ht','winner_age',
       'loser_ht','loser_age', 'winner_rank_points',  'loser_rank_points', 'winner_win%', 'loser_win%','winner_h2h%', 'winner_surface%', 'loser_surface%','winner_year%','loser_year%','winner_tournament%','loser_tournament%', 'winner_10%','loser_10%']]
y_train = matches_train['better_rank_win']

In [26]:
X_test = matches_test[['winner_ht','winner_age',
       'loser_ht','loser_age', 'winner_rank_points',  'loser_rank_points', 'winner_win%', 'loser_win%','winner_h2h%', 'winner_surface%', 'loser_surface%','winner_year%','loser_year%','winner_tournament%','loser_tournament%', 'winner_10%','loser_10%']]
y_test = matches_test['better_rank_win']

In [27]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
#trazenje hiperparametara
grid_model = ensemble.RandomForestRegressor()

In [29]:
hyper_par = {
    'n_estimators': range(30,40), 
    'max_depth': range(1, 20)
}

In [30]:
grid = model_selection.GridSearchCV(grid_model, param_grid=hyper_par,cv=3, scoring="r2",n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_score_)
print('n_estimators: ', grid.best_params_['n_estimators'], 
      'max_depth: ', grid.best_params_['max_depth'])

0.9779307325980989
n_estimators:  38 max_depth:  16


In [31]:
#primena slucajnih suma sa dobijenim hiperparametrima
model_forest = ensemble.RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], 
                                       max_depth=grid.best_params_['max_depth'])

In [32]:
model_forest.fit(X_train, y_train)
y_test_predicted = model_forest.predict(X_test)
y_train_predicted = model_forest.predict(X_train)

In [33]:
print('Test rezulati')
print('r2_score: ', metrics.r2_score(y_test, y_test_predicted))
print('RMSE: ', metrics.mean_squared_error(y_test, y_test_predicted, squared=False))

Test rezulati
r2_score:  0.9808211011813587
RMSE:  0.0678525285228978


In [34]:
print('Trening rezultati')
print('r2_score: ', metrics.r2_score(y_train, y_train_predicted))
print('RMSE: ', metrics.mean_squared_error(y_train, y_train_predicted, squared=False))

Trening rezultati
r2_score:  0.9973356299972708
RMSE:  0.02496379927519656


In [39]:
X = matches[['winner_ht','winner_age', 'loser_ht','loser_age', 'winner_rank_points',  'loser_rank_points', 'winner_win%', 'loser_win%','winner_h2h%', 'winner_surface%', 'loser_surface%','winner_year%','loser_year%','winner_tournament%','loser_tournament%', 'winner_10%','loser_10%']]
y = matches['better_rank_win']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.33)

In [40]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
grid_model = ensemble.RandomForestRegressor()
grid = model_selection.GridSearchCV(grid_model, param_grid=hyper_par,cv=3, scoring="r2",n_jobs=-1)

grid.fit(X_train, y_train)

In [ ]:
print(grid.best_score_)
print('n_estimators: ', grid.best_params_['n_estimators'], 
      'max_depth: ', grid.best_params_['max_depth'])

In [ ]:
model_forest.fit(X_train, y_train)
y_test_predicted = model_forest.predict(X_test)
y_train_predicted = model_forest.predict(X_train)

In [ ]:
print('Test rezulati')
print('r2_score: ', metrics.r2_score(y_test, y_test_predicted))
print('RMSE: ', metrics.mean_squared_error(y_test, y_test_predicted, squared=False))

In [ ]:
print('Trening rezultati')
print('r2_score: ', metrics.r2_score(y_train, y_train_predicted))
print('RMSE: ', metrics.mean_squared_error(y_train, y_train_predicted, squared=False))